# ATP Synthase Model attempt 7: appropriate parameters with efflux pump
7.29.2020 <br>
In this notebook we will add correct parameters to the simulation with the efflux pump.
---

In [1]:
# Import cell (refined)
import numpy as np
import pandas as pd 

from biocrnpyler import *
from bioscrape import *
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model

import colorcet

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row

bokeh.io.output_notebook()

Loading BokehJS ...

## Model ATP Synthase, and Proton Pump (no membrane integration)
### biocrnpyler CRNs

---
```
if ph outside <= 8: (ATP synthesis)
    ADP + Pi (+ 4 H+) --> (k1) ATP
    ATP --> (k2) ADP + Pi
    outside_H --> (k3) H+
    
```
----
From proton pump:
```
H+ -->(k7_efflux) outside_H
```
---
### Look at rate parameters

Some rate justifications: 
- k1 = ATP synthesis rate is > 100 ATP/sec (bionum ID 111975) or 6,000 ATP/min or 360,000 ATP/hr <br>
- k2 = ATP Use rate in Tx/Tl? - we normally assume there is none by hour 20<br>
- k3 = outside_H to H rate. Each ATP requires the transport or 3.33 protons, we round to 4. So 400 protons flow through / sec<br>
- k7_efflux = H to outside_H rate. Keep 400 protons / sec for to begin with (until more accurate mechanism is decided on)
---

In [2]:
species = ['atp','adp', 'pi', 'h', 'outside_h']
parameters = [('k1', 100*60*60), ('k2', 0.00008*60*60), ('k3', 400*60*60), ('k7_efflux', 400*60*60)]

rxn1 = (['adp', 'pi'], ['atp'], 'massaction', {'k':'k1'})
rxn2 = (['atp'], ['adp', 'pi'], 'massaction', {'k':'k2'})
rxn3 = (['outside_h'], ['h'], 'massaction', {'k':'k3'})
rxn4 = (['h'], ['outside_h'], 'massaction', {'k':'k7_efflux'})

reactions = [rxn1, rxn2, rxn3, rxn4]

reactions_noatpsyn = [rxn2]



x_0 = {
    
    'atp':30,
    'outside_h':15
}


M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)

M_noatpsyn =  Model(species = species, reactions = reactions_noatpsyn, parameters = parameters,
         initial_condition_dict = x_0)


timepoints = np.linspace(0,30,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)
df_results_noatpsyn = py_simulate_model(timepoints, Model = M_noatpsyn, stochastic = False)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, h, 
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, h, 


In [3]:
palette = colorcet.b_glasbey_category10

p = bokeh.plotting.figure(width = 500, height = 400, 
                             x_axis_label = 'time (hr)',
                             y_axis_label = 'concentration (mM)',
                             title = 'WITH ATP SYNTHASE',
                         )

p1 = bokeh.plotting.figure(width = 500, height = 400, 
                             x_axis_label = 'time (hr)',
                             y_axis_label = 'concentration (mM)',
                             title = 'WITHOUT ATP SYNTHASE')


p.line(timepoints, df_results['atp'], color = palette[0], line_width = 2,legend_label = 'atp')
p.line(timepoints, df_results['adp'], color = palette[1],line_width = 2, legend_label = 'adp')
p.line(timepoints, df_results['outside_h'], color = palette[4], line_width = 2,legend_label = 'outside_h')
p.line(timepoints, df_results['h'], color = palette[3], line_width = 2,legend_label = 'h')

p1.line(timepoints, df_results_noatpsyn['atp'], color = palette[0], line_width = 2,legend_label = 'atp')

p.legend.click_policy = 'hide'
bokeh.io.show(row(p,p1))


Ok, so if we put some accurate rate parameters in here, we see that atp is maintained fast! <br>
**Next steps: incorporate membrane integration with sub sbml. Get accurate rate parameters / mechanism for proton gradient mechanism. Incorporate degradation of membrane proteins (and eventual end of need of atp?)**

In [48]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,pandas,bokeh,colorcet,bioscrape,biocrnpyler,jupyterlab

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
pandas 1.0.3
bokeh 2.0.2
colorcet 2.0.2
bioscrape 1.0.0
biocrnpyler unknown
jupyterlab 1.2.6
